# Simularium Conversion Tutorial : PhysiCell Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.physicell import PhysicellConverter, PhysicellData
from simulariumio import UnitData, MetaData

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `PhysicellConverter` consumes discrete cell data from PhysiCell. 

The converter requires a `PhysicellData` object as a parameter.

`PhysicellData` contains the following:
* **meta_data** : `MetaData`
    * An object containing metadata for the trajectory including box size, scale factor, and camera defaults   
* **timestep** : *float*
    * A float amount of time in seconds that passes each step
    * Default: 0.0          
* **path_to_output_dir** : *string*
    * A string path to the PhysiCell output directory containing MultiCellDS XML and MATLAB files          
* **types** : *Dict\[int, Dict\[Any, str\]\] (optional)*
    * \[cell type ID from PhysiCell data\] : Dict\[Any, str\]
        * the cell type ID from PhysiCell data mapped to display name for that type, and display names for phases of that type
            * "name" or \[cell phase ID from PhysiCell data\] : str
                * "name" or the cell phase ID from PhysiCell data mapped to the display names
                * Default: "cell\[cell type ID from PhysiCell data\]#phase\[cell phase ID from PhysiCell data\]"
* **time_units**: `UnitData` *(optional)*
    * multiplier and unit name for time values
    * Default: 1.0 second
* **plots** : *List[Dict[str, Any]] (optional)*
    * An object containing plot data already in Simularium format
    
`MetaData` contains the following:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
        * *for now we only support rectangular bounds, but will be adding support for spheres soon*
* **camera_defaults**: `CameraData` (optional)
    * camera's initial settings which it also returns to when reset
* **scale_factor** : float (optional)
    * A multiplier for the scene, use if visualization is too large or small
    * Default: 1.0
    
`CameraData` contains the following:
* **position** : *np.ndarray (shape = \[3\]) (optional)*
    * 3D position of the camera itself
    * Default: np.array(\[0.0, 0.0, 120.0\])
* **look_at_position**: *np.ndarray (shape = \[3\]) (optional)*
    * position at which the camera looks
    * Default: np.zeros(3)
* **up_vector**: *np.ndarray (shape = \[3\]) (optional)*
    * the vector that defines which direction is "up" in the camera's view
    * Default: np.array(\[0.0, 1.0, 0.0\])
* **fov_degrees**: *float (optional)*
    * the angle defining the extent of the 3D world that is seen from bottom to top of the camera view
    * Default: 75.0

`UnitData` contains the following:
* **name**: *str*
    * unit name for values (we support this list https://github.com/hgrecco/pint/blob/master/pint/default_en.txt)
* **magnitude**: *float (optional)*
    * multiplier for values (in case they are not given in whole units)
    * Default: 1.0

In [2]:
box_size = 1000.0

example_data = PhysicellData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, 100.0]),
        scale_factor=0.01,
    ),
    timestep=360.0,
    path_to_output_dir="../simulariumio/tests/data/physicell/output/",
    types={
        0 : {
            "name" : "tumor cell",
            4 : "G0G1",
        },
        1 : {
            "name" : "motile tumor cell",
            4 : "G0G1",
        }
    },
    time_units=UnitData("s"),  # seconds
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
PhysicellConverter(example_data).write_JSON("example_physicell")

Reading PhysiCell Data -------------
Reading ../simulariumio/tests/data/physicell/output/output00000000.xml
Reading ../simulariumio/tests/data/physicell/output/output00000000_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000001.xml
Reading ../simulariumio/tests/data/physicell/output/output00000001_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000002.xml
Reading ../simulariumio/tests/data/physicell/output/output00000002_cells_physicell.mat
Writing JSON -------------
Converting Trajectory Data -------------
saved to example_physicell.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.